In [5]:
!pip install selenium
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ce/cd/a7c2cbffe2afff975349e60b14b63a448162145a7acac8ba12ddc2ed78a8/pandas-2.1.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.22.4 from https://files.pythonhosted.org/packages/cc/05/ef9fc04adda45d537619ea956bc33489f50a46badc949c4280d8309185ec/numpy-1.26.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.1 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 61.1/61.1 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   -----

In [2]:
#=================================importing necessary Libraries==============================================#
import time
import pandas as pd    
# ------------- # 
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains #to scrolldown 
import math
import re
import time

#=================================starting the program=========================================================#
start_time = time.strftime("%H:%M:%S", time.localtime())
print("Starting_Time: ",start_time)

#===========================giving User Credentials, roles, location, etc================================#
email = "email_@outlook.com"
password = "Password"
position = "data engineer intern"
location = "india"
#==================================this function is for starting the page=====================================#
def starting_page(email,password,position,location): #returning the driver and no of pages
    # Driver path
    driver = webdriver.Chrome()  

    # Maximize Window
    driver.maximize_window() 
    driver.minimize_window()  
    driver.maximize_window()  
    driver.switch_to.window(driver.current_window_handle)
    driver.implicitly_wait(10)

    # Enter to the site
    driver.get('https://www.linkedin.com/login');
    time.sleep(2)

    # Accept cookies
    
    #==============================finding the elements of username & password and sign in=======================#
    driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(email)
    driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(password)
    time.sleep(1)
    # Login button
    driver.find_element(By.XPATH,"//button[@aria-label='Sign in']").click()
    driver.implicitly_wait(10)

    #====================================going to the Jobs page==================================================#
    driver.get('https://www.linkedin.com/jobs/search')
    ## waiting load
    time.sleep(2)

    # Go to search results directly
    #==============fill job title, clearing the location placeholder and fill location and submit================#
    driver.find_element(By.XPATH,'//*[@aria-label="Search by title, skill, or company"]').send_keys(position)
    driver.find_element(By.XPATH,'//*[@aria-label="City, state, or zip code"]').clear()
    driver.find_element(By.XPATH,'//*[@aria-label="City, state, or zip code"]').send_keys(location)
    driver.find_element(By.CLASS_NAME,'jobs-search-box__submit-button').click()
    time.sleep(30)

    ##=============================clicking the date posted buttons and selecting past week=========================##
    date_posted_xpath = "//button[@aria-label='Date posted filter. Clicking this button displays all Date posted filter options.']"
    driver.find_element(By.XPATH,date_posted_xpath).click()
    time.sleep(30)
    driver.find_element(By.XPATH,"//span[text()='Past week']").click()
    #=======show results button - multiple show results buttons present - taking the 1st search button for now========#
    list_date_modified = driver.find_elements(By.CSS_SELECTOR,'div.reusable-search-filters-buttons>button:nth-child(2)>span')
    list_date_modified[0].click()
    time.sleep(10)

    #==============all buttons in the options - jobs, date posted, experience level, company, on-site remote==============#
    all_buttions =driver.find_elements(By.CSS_SELECTOR,"button.artdeco-pill:nth-child(1)")
    #took all the buttons and selected the 3rd one for experience level

    #=====click the experience level and click on entry level, internship and click on show results buttons============#
    all_buttions[2].click() #clicking the experience level button
    time.sleep(20)
    driver.find_element(By.XPATH,"//span[text()='Internship']").click()
    time.sleep(3)
    driver.find_element(By.XPATH,"//span[text()='Entry level']").click()
    time.sleep(3)
    #show results button - multiple show results buttons present - taking the 2nd search button for now
    list_date_modified = driver.find_elements(By.CSS_SELECTOR,'div.reusable-search-filters-buttons>button:nth-child(2)>span')
    list_date_modified[1].click()
    time.sleep(10)
    
    #==========================getting the total results to calculate the no of pages================================#
    total_results_text = driver.find_element(By.CSS_SELECTOR,'div.jobs-search-results-list__subtitle').text
    print(total_results_text)
    total_results= int("".join(re.findall(r'\d+', total_results_text)))
    
    #=======================================calculating the no of pages===============================================#
    total_pages = math.ceil(int(total_results)/25)
    print(total_pages)
 
    return driver, total_pages #returning driver and total pages

functioncall = starting_page(email,password,position,location)
driver = functioncall[0]
total_pages = functioncall[1]

#========================Navigate all pages and collecting all links=============================================#
print('Links are being collected now. So go and minimize the screen to collect all links')
#=========================screen has to minimized here===============================================#

def collect_links(driver,total_pages):#giving driver and total pages

    links = []

    try: 
        
        #========================going to the Jobs page==================================================#
        for page in range(2,total_pages+1):
            time.sleep(2)
            #==========================identify and get entire jobs links block=================#
            jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
            
            #============================get all job linkselements==============================#
            jobs_list= jobs_block.find_elements(By.CLASS_NAME, 'job-card-list__entity-lockup')
            
            for i in jobs_list:
                driver.execute_script("arguments[0].scrollIntoView();", i)
                time.sleep(2)
            #print(jobs_list)  
                jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
                jobs_list= jobs_block.find_elements(By.CLASS_NAME, 'job-card-list__entity-lockup')
            
            #==========================identify and get entire jobs links block=================#
            #============================get all job linkselements==============================#
            jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
            jobs_list= jobs_block.find_elements(By.CLASS_NAME, 'job-card-list__entity-lockup')
            
            #print(jobs_block.find_elements(By.XPATH,all_links_xpath))
            #=====================collect the links one by one===================================#
            for job in jobs_list:
                all_links = job.find_elements(By.TAG_NAME,'a')
                #print(all_links)
                for a in all_links:
                    if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
                        links.append(a.get_attribute('href'))
                    else:
                        pass
                # scroll down for each job element
                driver.execute_script("arguments[0].scrollIntoView();", job)
                driver.implicitly_wait(10)
            
            #==========================going to the next page and prining the current time=============#
            curr_time = time.strftime("%H:%M:%S", time.localtime())
            print(f'Collecting the links in the page: {page-1}',"Current Time is :", curr_time)
            
            #==========================going to 8th page and giving refresh=============================#
            if page==8:
                driver.refresh()
                time.sleep(20)
            #===================find the next page element and click to next page=======================#
            driver.find_element(By.XPATH,f"//button[@aria-label='Page {page}']").click()
            time.sleep(3)
            
    except:
        pass

    return links

links = collect_links(driver,total_pages)

print('Found ' + str(len(links)) + ' links for job offers')
driver.close()

pd.DataFrame({'Links':links}).to_csv(r'D:\scrapped_jobs\Oct22_LinkedinExtractedLinks.csv',index=False)
links = pd.read_csv(r'D:\scrapped_jobs\Oct22_LinkedinExtractedLinks.csv').Links.to_list()

Starting_Time:  15:57:43
687 results
28
Links are being collected now.
Found 50 links for job offers


In [3]:
#==============================creating lists to append all the elements form csv================================#
import datetime
# for timezone()
import pytz
Job_Overall_Details = {"job_titles":[],"contact_person":[],"contact_team":[],"company_name":[],"company_link": [],
                       "location_name":[],"employmemt_type": [],"seniority_level":[],"post_date":[],
                       "applicants_count":[],"description":[],"job_links":[]}

wait_point = 1

In [10]:
#print(len(job_titles),len(contact_person),len(contact_team),len(company_name),len(company_link),len(location_name),len(employmemt_type),len(seniority_level),len(post_date),len(applicants_count),len(description),len(job_links))
print(wait_point)
print(links[4])
print(Job_Overall_Details["job_titles"][4])

1
https://www.linkedin.com/jobs/view/3728816636/?eBP=JOB_SEARCH_ORGANIC&refId=vig3LjUN0urbHtcobLSQPA%3D%3D&trackingId=SQH7ZoOXQoJkOV6NBPfSbA%3D%3D&trk=flagship3_search_srp_jobs
Hadoop Cluster Developer


In [12]:
#passing the links and get the details of each job and taking the contacts
start_time = time.strftime("%H:%M:%S", time.localtime())
print('Start time:', start_time)
#
def JobDetailedScrapper(links,Job_Overall_Details,wait_point):#giving links and overall details dict and wait point
    #==============================collecting the details of each job===========================================#
    for link in links:
        #======================checking if the job is already scrapped=========================================#
        if link not in Job_Overall_Details["job_links"]:
            #==============================creating the chrome driver===========================================#
            op = webdriver.ChromeOptions()
            op.add_argument('headless')
            driver = webdriver.Chrome(options=op)
            # driver = webdriver.Chrome()
            driver.get(link)
            driver.maximize_window()
            time.sleep(2)
            # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # time.sleep(1)
            #=============================clicking the show more button=========================================#
            driver.find_element(By.CLASS_NAME,"show-more-less-html__button").click()
            #=============================getting the top card job details==========================================#
            top_card = driver.find_element(By.CLASS_NAME,'top-card-layout__card')
            
            #=============================getting the all job details===============================================#
            
            Job_Overall_Details["job_titles"].append(top_card.find_element(By.TAG_NAME,'h1').text)
            
            #=====================trying to fetch the contact person and contact team if available==================#
            try:
                person = driver.find_element(By.CSS_SELECTOR,"div.message-the-recruiter>div>div>h3").text
                team = driver.find_element(By.CSS_SELECTOR,"div.message-the-recruiter>div>div>h4").text
                print(person,team)
                Job_Overall_Details["contact_person"].append(person)
                Job_Overall_Details["contact_team"].append(team)
            except:
                Job_Overall_Details["contact_person"].append('')
                Job_Overall_Details["contact_team"].append('')
                
            #=============================getting the company details===============================================#
            Job_Overall_Details["company_name"].append(top_card.find_element(By.CSS_SELECTOR,'div.top-card-layout__entity-info>h4>div>span').text)
            Job_Overall_Details["company_link"].append(top_card.find_element(By.TAG_NAME,'a').get_attribute('href'))
            Job_Overall_Details["location_name"].append(top_card.find_element(By.CSS_SELECTOR,'div.top-card-layout__entity-info>h4>div>span.topcard__flavor--bullet').text)
            
            #=============================getting the post date and applicant count=================================#
            Job_Overall_Details["post_date"].append(top_card.find_element(By.CLASS_NAME,'posted-time-ago__text').text)
            Job_Overall_Details["applicants_count"].append(top_card.find_element(By.CLASS_NAME,'num-applicants__caption').text)
            
            #=============================getting the seniority level and employment type============================#
            job_criteria_text_list = [i.text for i in driver.find_elements(By.CSS_SELECTOR,"span.description__job-criteria-text")]
            
            #employment type
            if len(job_criteria_text_list) >= 2:
                Job_Overall_Details["employmemt_type"].append(job_criteria_text_list[1])
            else:
                Job_Overall_Details["employmemt_type"].append('None')
            
            Job_Overall_Details["seniority_level"].append(job_criteria_text_list[0])
            
            #=============================getting the job description and job links==================================#
            #print(driver.find_element(By.CLASS_NAME,'description__text').text)
            Job_Overall_Details["description"].append(driver.find_element(By.CLASS_NAME,'description__text').text)
            Job_Overall_Details["job_links"].append(link)
            
            #===================================closing the driver==================================================#
            driver.close()

            wait_point += 1
            #=================================if 15 jobs scrapped then sleep for 10 sec===========================#
            if wait_point % 15 == 0:
                
                #using now() to get current time
                current_time = time.strftime("%H:%M:%S", time.localtime())
                print(f"{wait_point} links completed"," The current time in india is :", current_time)
                time.sleep(10)
                
        else:
            
            pass
    
    return Job_Overall_Details

#=============================calling the function===============================================# 
JobScrapperReturn = JobDetailedScrapper(links,Job_Overall_Details,wait_point)       
df = pd.DataFrame(JobScrapperReturn)

df.to_csv(r'D:\scrapped_jobs\Oct22_data_engineer_intern_linkedin_jobs.csv',index=False)
df.head()


Start time: 17:00:14


,job_titles,contact_person,contact_team,company_name,company_link,location_name,employmemt_type,seniority_level,post_date,applicants_count,description,job_links
0,Statistical Programmer,,,Pfizer,https://www.linkedin.com/company/pfizer?trk=pu...,"Chennai, Tamil Nadu, India",Full-time,Entry level,46 minutes ago,Over 200 applicants,Job Summary:\n\nThis role is a hands on Statis...,https://www.linkedin.com/jobs/view/3693559357/...
1,Statistical Programmer,,,Pfizer,https://www.linkedin.com/company/pfizer?trk=pu...,"Chennai, Tamil Nadu, India",Full-time,Entry level,47 minutes ago,Over 200 applicants,Job Summary:\n\nThis role is a hands on Statis...,https://www.linkedin.com/jobs/view/3693559357/...
2,Outsystems Development,,,UST,https://www.linkedin.com/company/ustglobal?trk...,"Kochi, Kerala, India",Full-time,Entry level,6 days ago,29 applicants,Job Description\n\nRole Proficiency:\n\nAct cr...,https://www.linkedin.com/jobs/view/3741790299/...
3,Big Data Developer,Subhashis Sahoo,Director at Avihs,Avihs,https://in.linkedin.com/company/www.avihs.com?...,"Bengaluru, Karnataka, India",Full-time,Entry level,5 days ago,74 applicants,We are seeking a skilled Big Data Engineer / A...,https://www.linkedin.com/jobs/view/3736279833/...
4,Hadoop Cluster Developer,,,Infosys,https://in.linkedin.com/company/infosys?trk=pu...,"Trivandrum, Kerala, India",Full-time,Entry level,2 days ago,50 applicants,"Primary skills:Analytics->Cluster Analysis,App...",https://www.linkedin.com/jobs/view/3730473721/...


In [10]:
df = pd.DataFrame(JobScrapperReturn)

df.to_csv(r'D:\scrapped_jobs\Oct22_data_engineer_intern_intern_22ndOCT_22_linkedin_jobs.csv',index=False)
df.head()


,job_title,contact_person,company_name,company_link,location_name,post_dates,applicants_count,employment_type,seniority_level,description,Job_links
0,Analyst Data Science Analytics,,ClickJobs.io,https://uk.linkedin.com/company/clickjobsio?tr...,"Bengaluru, Karnataka, India",5 hours ago,103 applicants,Full-time,Entry level,Grow your career with a growing organization\n...,https://www.linkedin.com/jobs/view/3738351411/...
1,Data Analyst (Mutual Series),,Franklin Templeton India,https://in.linkedin.com/company/franklintemple...,"Mumbai, Maharashtra, India",2 days ago,Over 200 applicants,Full-time,Entry level,"At Franklin Templeton, everything we do is foc...",https://www.linkedin.com/jobs/view/3737214595/...
2,Process Developer – Data Analyst-COR021664,,Genpact,https://www.linkedin.com/company/genpact?trk=p...,"Noida, Uttar Pradesh, India",4 days ago,Over 200 applicants,Full-time,Entry level,"With a startup spirit and 90,000+ curious and ...",https://www.linkedin.com/jobs/view/3623351494/...
3,Forecasting Analyst,,NewAge Products Inc.,https://ca.linkedin.com/company/newage-product...,India,1 day ago,Over 200 applicants,Full-time,Entry level,"NewAge Products Inc. (NAP), headquartered in N...",https://www.linkedin.com/jobs/view/3737546187/...
4,Internal Audit - Data Strategy - Analyst - Hyd...,,Goldman Sachs,https://www.linkedin.com/company/goldman-sachs...,"Hyderabad, Telangana, India",4 hours ago,Over 200 applicants,Full-time,Entry level,Job Description\n\nWhat We Do\n\nInternal Audi...,https://www.linkedin.com/jobs/view/3648716404/...


In [13]:
#merging all the datasets and getting only contact
import pandas as pd
import glob
import numpy as np
import datetime

#function to read all csv files 
def read_file(): #reading files inside the folders
    
    csv_path = r'D:\scrapped_jobs' #

    # filenames = glob.glob(path + "\*.xlsx")
    path_filenames = glob. glob(csv_path + "/*.csv")
    path_filenames.sort()
#     print('File names:', csv_filenames)
    
    return path_filenames

AllFiles=read_file()

def file_names(path_filenames): #getting file names
    files = []
    
    for index,file in enumerate(path_filenames):
        f = file.split("D:\\scrapped_jobs\\")[1].split(".csv")[0]
        files.append(f'{f}')

    return files

files = file_names(read_file())

merged_frame = pd.DataFrame({'job_title':[], 'contact_person':[], 'company_name':[],'Job_links':[],'scrap_date':[]})

for i,j in zip(AllFiles,files):
    
    month_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    
    df = pd.read_csv(i) #creating dataframe by reading csv files
    
    #finding the scrap date and adding it to the dataframe
    scrap_day = j.split('_')[0]
    if scrap_day[0:3] in month_list:
        scrap_date = datetime.datetime(2023, month_list.index(scrap_day[0:3])+1, int(scrap_day[3:]))
        df['scrap_date'] = scrap_date
        columns_to_take = ['job_title', 'contact_person', 'company_name','Job_links','scrap_date']
    
        try:
            df = df.loc[:,columns_to_take]
            bool_index = df.contact_person.apply(lambda x: type(x).__name__) != 'float'
            index_list = [j for j,i in enumerate(bool_index) if i == True]
            df = df.iloc[index_list,[0,1,2,3,4]]
            merged_frame = pd.concat([merged_frame,df],ignore_index=True)
            
        except:
            pass
    
merged_frame.sort_values('scrap_date',inplace=True)
merged_frame.to_csv(r'D:\scrapped_jobs\LinkedinContactsData.csv',index=False)
